# Fully Connected Auto-Encoder wb Sweeps
### $Time$ $Series$ $3rd$ $Test$

$Vasco$ $Mergulhão$ $-$ $Jan$ $2023$

### Version 1:
 - Applies Weights and Biases Sweeps on a given Sub-Sample.
 - Only uses FC-AE architectures

### ANN Configurations:

- #### Architecture(s)
    - Fully Connected Auto Encoder
        - Small (Input, 200, 200, LatDim)
        - Medium (Input, 300, 300, 300, 300, LatDim)
        - N2D [other papers orig ref] (Input, 500, 500, 2000, LatDim)
    
- #### Hyperparamenters
    - Latent Space Size
    - Batch Size
        - Small test [2 - 32] and Large test [128 - 256]
    - Learning Rate
    - Learning Rate Scheduler
        - Performance Schedulling
    - Activation Functions
        - SELU and Leaky ReLU
    - Initializations
        - LeCun and He (accordingly)
    - Batch Normalization
        - With/Without tests (note: if data is not z-scored, SELU not worth it, downgrade to ELU)
    - Optimizers
        - Nadam and SDG(momentum [0.9], Nesterov)
    - Epochs
        - 100 with Early Stopping
 

---
# 

In [1]:
import os
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta, date
import time
import datetime
import scipy

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, LeakyReLU, Activation
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow import keras

import wandb
from wandb.keras import WandbCallback

from sklearn.model_selection import train_test_split

---
# Script Variables

In [2]:
# Defines Dataset for the Sweep
dataset_name = 'Rwanda_10k_Set_1_w90'

# Uses name to navigate folders
dataset_folder = "_".join(dataset_name.split('_')[:-1]) #Takes out window length section
dataset_location = f'../Data/{dataset_folder}/{dataset_name}.csv'

# Zcore Data Decision
zscore_data = True # Set to: [True/False]
zscore_data_done = False # Always set to False. Ensures its not normalized multiple times

# Model Name and Variables
AE_Model_Name = 'FC_N2D' # Options: FC_Small, FC_Medium, FC_N2D
latent_layer_size = 25

# Sweep Names and Configurations
Project_Name = f'FC_AE-{dataset_name}'
Sweep_Config = f'{AE_Model_Name}_v2'
sweep_count = 6
use_running_sweep_id = False # Set to: [True/False]
running_sweep_id = 'ikwjt20r'


In [3]:
def window_col_names(dataset_name, win_prefix = 'd'):
    # retriving window length
    window_len = int(dataset_name.split('_')[-1][1:]) # Gets _wXX part of name, then skips 'w' to get the number.
    # defining window column names
    window_cols = [None]*window_len
    for i  in range(window_len):
        window_cols[i] = f'{win_prefix}' + str(i+1)
        
    return window_cols

---
# Data Imports

In [4]:
Data = pd.read_csv(dataset_location)

In [5]:
Data.head()

,short_ID,window_ID,window_start_date,d1,d2,d3,d4,d5,d6,d7,...,d81,d82,d83,d84,d85,d86,d87,d88,d89,d90
0,9,0,2018-01-03,29.333333,28.333333,27.333333,614.732454,613.732454,24.333333,23.333333,...,9.333333,7.374942,7.333333,5.378137,5.333333,3.382164,2.386019,2.333333,1.333333,0.333333
1,9,1,2018-04-03,13.578796,12.587951,11.593588,10.603692,10.583356,9.583356,8.583356,...,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,6.682928,5.699398,5.708333,3.721609,2.729514
2,9,2,2018-07-02,2.708333,0.744861,0.708333,-7.000000,6.719641,5.719641,4.719641,...,3.599352,2.599352,1.599352,0.599352,-7.000000,7.657384,6.657384,5.657384,4.657384,3.657384
3,9,3,2018-09-30,2.657384,1.657384,0.657384,-7.000000,6.718287,5.718287,4.718287,...,7.715255,6.715255,5.715255,4.715255,3.715255,2.715255,1.715255,0.715255,-7.000000,-7.000000
4,9,4,2018-12-29,-7.000000,-7.000000,-7.000000,6.738970,5.738970,4.738970,3.738970,...,-7.000000,-7.000000,-7.000000,7.688646,6.688646,5.688646,4.688646,3.688646,2.688646,1.688646


---
# Z-Scoring Data

This is done on a row-by-row basis.<br>
Meaning, each window is normalized to its own Mean and Std.

In [6]:
def Zscore (df_in, all_neg_replace = -1):
    # retriving sets of columns
    window_cols = window_col_names(dataset_name)        
    cols = df_in.columns.to_list()
    other_cols = list(set(cols) - set(window_cols))
    
    # Zscore can't handle constant (flat) inputs
    # Therefore windows with all negative credit output NaNs
    # Instead these are excluded from zcoring and are manually scaled
    # Default scalling is  -7 => -1
    all_neg_replace = float(all_neg_replace)
    all_neg_index = df_in[(df_in[window_cols] == float(-7)).all(axis=1)].index
    positive_index = list(set(df_in.index.to_list()) - set(all_neg_index))
    
    if len(all_neg_index.to_list()) > 0:
        # Aux DF for replacing -7 with all_neg_replace (i.e, -1)
        df_allneg = df_in.iloc[all_neg_index].copy()
        df_allneg.loc[:, window_cols] = all_neg_replace

        # Zscoring only non-constant rows
        df_positive = df_in.iloc[positive_index].copy()    
        df_positive[window_cols] = scipy.stats.zscore(df_positive[window_cols],
                                                         axis=1,
                                                         nan_policy = 'omit')

        df_zscore = pd.concat([df_positive, df_allneg])
        df_zscore.sort_values(by=['short_ID', 'window_ID'], inplace = True)

        # If indeces match, then the join was successful
        if df_zscore.index.to_list() == df_in.index.to_list():
            return df_zscore
        # Otherwise raise error
        else:
            print('Error Zscoring')
    else:
        df_zscore = pd.DataFrame(columns= cols)
        df_zscore[window_cols] = scipy.stats.zscore(df_in[window_cols],
                                                         axis=1,
                                                         nan_policy = 'omit')
        df_zscore[other_cols] = df_in[other_cols]
        

    return df_zscore

In [7]:
if zscore_data == True and zscore_data_done == False:
    Data = Zscore(Data)
    zscore_data_done = True
    print('Data WAS Zscored')
    
elif zscore_data == True and zscore_data_done == True:
    print('Already WAS Zscored')
    
elif zscore_data == False:
    print('Data NOT Zscored')
    
else:
    print('Error')

Data WAS Zscored


---
# NaN Checks and Policy

In [8]:
def NaN_policy (df, method_fullrows = 'ffill', method_sparserows = 'ffill'):
    # This functions checks and corrects NaNs.
    # It has redundancy built into it to double check for NaNs.
    # And allows for different policies for sparse NaN values and rows full of NaNs.
    
    df = df.copy(deep=True)
    window_cols = window_col_names(dataset_name)
        
    if df[window_cols].isna().any().any():
        print('NaN values detected.')
        
        nan_exist = True
        i = 0   
        while nan_exist:
            
            # Makes sure the policy only iterates with one redundant round.
            if i >= 2:
                print('NaN Policy Failed')
                break            

            #Checking if there are rows with only NaNs
            if df[window_cols].isna().all(axis=1).any():
                n_nan_rows = len(df[df[window_cols].isna().all(axis=1)])
                print(f'There are {n_nan_rows} rows of all NaN values.')

                #Correcting only NaN rows 
                df.fillna(method= method_fullrows, axis = 0, inplace = True)
                if ~ df[window_cols].isna().all(axis=1).any():
                    print(f'Rows of NaN corrected')
                    if ~ df[window_cols].isna().any().any():
                        nan_exist = False
                        print(f'NaNs no longer detected.')
            
            #Checking if there are rows with any NaNs in them
            if df[window_cols].isna().any(axis=1).any():
                n_nan_rows = len(df[window_cols].isna().any(axis=1))
                print(f'There are {n_nan_rows} rows with some NaN values.')
                
                # Correcting NaN inside rows 
                df.fillna(method= method_sparserows, axis = 1, inplace = True)
                if ~ df[window_cols].isna().any(axis=1).any():
                    print(f'Rows with some NaN were corrected')
                    if ~ df[window_cols].isna().any().any():
                        nan_exist = False
                        print(f'NaNs no longer detected.')

            i += 1
            

    else:
        print('No NaNs detected')
        
    return df

In [9]:
Data = NaN_policy(Data)

No NaNs detected


---
# Train-Test Split

In [10]:
window_cols = window_col_names(dataset_name)
data_array = Data[window_cols].to_numpy()

In [11]:
x_train, x_test = train_test_split(data_array, test_size=0.2, random_state=42)

---
# WandB Sweep Config
https://github.com/wandb/examples/blob/master/colabs/keras/Keras_param_opti_using_sweeps.ipynb


In [12]:
wandb.login()

wandb: Currently logged in as: vasco-mergulhao (vasco-phd). Use `wandb login --relogin` to force relogin


True

---
---
# Model & Sweep Configurations

## Generic Training Functions

In [13]:
def get_initialization (activation_fn = 'SELU'):
    # Select right initialization for respective activation function between SELU and LeakyReLU
    if activation_fn.lower() == "selu":
        return 'lecun_normal'
    if activation_fn.lower() == "leakyrelu":
        return 'he_normal'

In [14]:
def get_activation_fn (activation_fn = 'SELU'):
    # Select right initialization for respective activation function between SELU and LeakyReLU
    if activation_fn.lower() == "selu":
        return Activation('selu')
    if activation_fn.lower() == "leakyrelu":
        return LeakyReLU(alpha=0.2)

In [15]:
def get_optimizer(lr=0.0001, optimizer="nadam"):
    # Select optmizer between adam and sgd
    if optimizer.lower() == "nadam":
        return tf.keras.optimizers.Nadam(learning_rate=lr, beta_1=0.9, beta_2=0.999)
    if optimizer.lower() == "sgd":
        return tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9, nesterov=True)

In [16]:
def train(model, batch_size= 32, epochs= 100, lr=0.001, optimizer='nadam'):  
    
    tf.keras.backend.clear_session()
    model.compile(loss="mse", 
                  optimizer=get_optimizer(lr, optimizer), 
                  metrics=["mse"])

    early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
    lr_scheduler_cb = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)

    model.fit(x_train, 
              x_train, 
              batch_size=batch_size, 
              epochs=epochs, 
              validation_data=(x_test, x_test), 
              callbacks=[WandbCallback(), early_stopping_cb, lr_scheduler_cb])
    
    

In [17]:
def sweep_train(config_defaults=None):
    # Initialize wandb with a sample project name
    with wandb.init(config=config_defaults):  # this gets over-written in the Sweep

        # Specify the other hyperparameters to the configuration
        wandb.config.architecture_name = AE_Model_Name
        wandb.config.dataset_name = dataset_name

        # initialize model
        if AE_Model_Name == 'FC_Small':
            AE_model = Small_AE(wandb.config.window_length,
                            wandb.config.latent_layer_size,
                            wandb.config.activation_fn)
        if AE_Model_Name == 'FC_Medium':
            AE_model = Medium_AE(wandb.config.window_length,
                            wandb.config.latent_layer_size,
                            wandb.config.activation_fn)
            
        if AE_Model_Name == 'FC_N2D':
            AE_model = N2D_AE(wandb.config.window_length,
                            wandb.config.latent_layer_size,
                            wandb.config.activation_fn)    
            

        train(AE_model, 
              wandb.config.batch_size, 
              wandb.config.epochs,
              wandb.config.learning_rate,
              wandb.config.optimizer)
        


## FC_Small
- (Input, 200, 200, Enconder)

### Sweep Configuration

In [18]:
if AE_Model_Name == 'FC_Small':
    
    sweep_config = {
    'method': 'random',
    'name': Sweep_Config,
    }
    
    metric = {
    'name': 'mse',
    'goal': 'minimize'
    }
    sweep_config['metric'] = metric
    
    parameters_dict = {
        'optimizer': {
            'values': ['nadam', 'sgd']
        },
        'latent_layer_size': {
            'value': latent_layer_size #previous [values: [5, 10, 25]
        },
        'epochs':{
            'value': 100
        },
        'window_length':{
            'value': int(dataset_name.split('_')[-1][1:])
        },
        'activation_fn':{
            'values': ['SELU','LeakyReLU']
        }
    }
    sweep_config['parameters'] = parameters_dict
    
    parameters_dict.update({
        'learning_rate': {
            # a flat distribution between 0 and 0.1
            'distribution': 'log_uniform_values',
            'min': 0.001,
            'max': 0.1,
          },
        'batch_size': {
            # integers between 2 and 256
            # with evenly-distributed logarithms 
            'distribution': 'q_log_uniform_values',
            'q': 2,
            'min': 2,
            'max': 256,
          }
        })
    print('Code Used')
else:
    print('NOT Active')

NOT Active


### _Model_: FC_Small

In [19]:
def Small_AE(window_length = 90, latent_layer_size = 5, activation_fn = 'SELU'):
    
    inputs = Input(shape= window_length)
    
    layer_e1 = Dense(200, activation = get_activation_fn(activation_fn),
                     kernel_initializer=get_initialization(activation_fn))(inputs)
    
    layer_e2 = Dense(200, activation = get_activation_fn(activation_fn),
                     kernel_initializer=get_initialization(activation_fn))(layer_e1)
    #Latent Space (no activation)
    encoded = Dense(latent_layer_size)(layer_e2)
       
    layer_d1 = Dense(200, activation = get_activation_fn(activation_fn),
                     kernel_initializer=get_initialization(activation_fn))(encoded)
    
    layer_d2 = Dense(200, activation = get_activation_fn(activation_fn),
                     kernel_initializer=get_initialization(activation_fn))(layer_d1)
    
    decoded = Dense(window_length)(layer_d2)
       
    autoencoder = keras.models.Model(inputs=inputs, outputs = decoded)
    
    return autoencoder   

In [20]:
if AE_Model_Name == 'FC_Small':
    AE_model = Small_AE(latent_layer_size = latent_layer_size)
    AE_model.summary()
    print('Code Used')
else:
    print('NOT Active')    

NOT Active


---
## FC_Medium
-  (Input, 300, 300, 300, 300, Encoder)

### Sweep Configuration

In [21]:
if AE_Model_Name == 'FC_Medium':
    
    sweep_config = {
    'method': 'random',
    'name': Sweep_Config,
    }
    
    metric = {
    'name': 'mse',
    'goal': 'minimize'
    }
    sweep_config['metric'] = metric

    parameters_dict = {
        'optimizer': {
            'values': ['nadam', 'sgd']
        },
        'latent_layer_size': {
            'value': latent_layer_size #previous [values: [5, 10, 25]
        },
        'epochs':{
            'value': 100
        },
        'window_length':{
            'value': int(dataset_name.split('_')[-1][1:])
        },
        'activation_fn':{
            'values': ['SELU','LeakyReLU']
        }
    }
    sweep_config['parameters'] = parameters_dict
    
    parameters_dict.update({
        'learning_rate': {
            # a flat distribution between 0 and 0.1
            'distribution': 'log_uniform_values',
            'min': 0.00001, #v_1 = 0.001
            'max': 0.001, #v_1 = 0.1
          },
        'batch_size': {
            # integers between 2 and 256
            # with evenly-distributed logarithms 
            'distribution': 'q_log_uniform_values',
            'q': 2,
            'min': 20, #v_1 = 2
            'max': 256,
          }
        })
    print('Code Used')
else:
    print('NOT Active')

NOT Active


### _Model_: FC_Medium

In [22]:
def Medium_AE(window_length = 90, latent_layer_size = 5, activation_fn = 'SELU'):
    
    inputs = Input(shape= window_length)
    
    layer_e1 = Dense(300, activation = get_activation_fn(activation_fn),
                     kernel_initializer=get_initialization(activation_fn))(inputs)    
    layer_e2 = Dense(300, activation = get_activation_fn(activation_fn),
                     kernel_initializer=get_initialization(activation_fn))(layer_e1)    
    layer_e3 = Dense(300, activation = get_activation_fn(activation_fn),
                     kernel_initializer=get_initialization(activation_fn))(layer_e2)    
    layer_e4 = Dense(300, activation = get_activation_fn(activation_fn),
                     kernel_initializer=get_initialization(activation_fn))(layer_e3)
    #Latent Space (no activation)
    encoded = Dense(latent_layer_size)(layer_e4)
       
    layer_d1 = Dense(300, activation = get_activation_fn(activation_fn),
                     kernel_initializer=get_initialization(activation_fn))(encoded)
    layer_d2 = Dense(300, activation = get_activation_fn(activation_fn),
                     kernel_initializer=get_initialization(activation_fn))(layer_d1)
    layer_d3 = Dense(300, activation = get_activation_fn(activation_fn),
                     kernel_initializer=get_initialization(activation_fn))(layer_d2)
    layer_d4 = Dense(300, activation = get_activation_fn(activation_fn),
                     kernel_initializer=get_initialization(activation_fn))(layer_d3)    
    decoded = Dense(window_length)(layer_d4)
       
    autoencoder = keras.models.Model(inputs=inputs, outputs = decoded)
    
    return autoencoder   

In [23]:
if AE_Model_Name == 'FC_Medium':
    AE_model = Medium_AE(latent_layer_size = latent_layer_size)
    AE_model.summary()
    print('Code Used')
else:
    print('NOT Active')

NOT Active


---
## FC_N2D
-  (Input, 500, 500, 2000, LatDim)

### Sweep Configuration

In [24]:
if AE_Model_Name == 'FC_N2D':
    
    sweep_config = {
    'method': 'random',
    'name': Sweep_Config,
    }
    
    metric = {
    'name': 'mse',
    'goal': 'minimize'
    }
    sweep_config['metric'] = metric

    parameters_dict = {
        'optimizer': {
            'values': ['nadam', 'sgd']
        },
        'latent_layer_size': {
            'value': latent_layer_size #previous [values: [5, 10, 25]
        },
        'epochs':{
            'value': 100
        },
        'window_length':{
            'value': int(dataset_name.split('_')[-1][1:])
        },
        'activation_fn':{
            'values': ['SELU','LeakyReLU']
        }
    }
    sweep_config['parameters'] = parameters_dict
    
    parameters_dict.update({
        'learning_rate': {
            # a flat distribution between 0 and 0.1
            'distribution': 'log_uniform_values',
            'min': 0.00001, #v_1 = 0.001
            'max': 0.01, #v_1 = 0.1
          },
        'batch_size': {
            # integers between 2 and 256
            # with evenly-distributed logarithms 
            'distribution': 'q_log_uniform_values',
            'q': 2,
            'min': 20, #v_1 = 2
            'max': 256,
          }
        })
    print('Code Used')
else:
    print('NOT Active')

Code Used


### _Model_: FC_N2D

In [25]:
def N2D_AE(window_length = 90, latent_layer_size = 5, activation_fn = 'SELU'):
    
    inputs = Input(shape= window_length)
    
    layer_e1 = Dense(500, activation = get_activation_fn(activation_fn),
                     kernel_initializer=get_initialization(activation_fn))(inputs)    
    layer_e2 = Dense(500, activation = get_activation_fn(activation_fn),
                     kernel_initializer=get_initialization(activation_fn))(layer_e1)    
    layer_e3 = Dense(2000, activation = get_activation_fn(activation_fn),
                     kernel_initializer=get_initialization(activation_fn))(layer_e2)    
    #Latent Space (no activation)
    encoded = Dense(latent_layer_size)(layer_e3)
       
    layer_d1 = Dense(2000, activation = get_activation_fn(activation_fn),
                     kernel_initializer=get_initialization(activation_fn))(encoded)
    layer_d2 = Dense(500, activation = get_activation_fn(activation_fn),
                     kernel_initializer=get_initialization(activation_fn))(layer_d1)
    layer_d3 = Dense(500, activation = get_activation_fn(activation_fn),
                     kernel_initializer=get_initialization(activation_fn))(layer_d2)
    # Output Layer, no activation function
    decoded = Dense(window_length)(layer_d3)
       
    autoencoder = keras.models.Model(inputs=inputs, outputs = decoded)
    
    return autoencoder   

In [26]:
if AE_Model_Name == 'FC_N2D':
    AE_model = N2D_AE(latent_layer_size = latent_layer_size)
    AE_model.summary()
    print('Code Used')
else:
    print('NOT Active')

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 90)]              0         
                                                                 
 dense (Dense)               (None, 500)               45500     
                                                                 
 dense_1 (Dense)             (None, 500)               250500    
                                                                 
 dense_2 (Dense)             (None, 2000)              1002000   
                                                                 
 dense_3 (Dense)             (None, 25)                50025     
                                                                 
 dense_4 (Dense)             (None, 2000)              52000     
                                                                 
 dense_5 (Dense)             (None, 500)               100050

---
---
# Run Sweep

In [27]:
if use_running_sweep_id == True:
    sweep_id = running_sweep_id
    print(f'Using sweep ID: {sweep_id}')
    print(f'Sweep URL: https://wandb.ai/vasco-phd/{Project_Name}/sweeps/{sweep_id}')
else:
    sweep_id = wandb.sweep(sweep_config, project = Project_Name)

Create sweep with ID: hb8t9ucw
Sweep URL: https://wandb.ai/vasco-phd/FC_AE-Rwanda_10k_Set_1_w90/sweeps/hb8t9ucw


In [28]:
wandb.agent(sweep_id, function=sweep_train, count= sweep_count)

wandb: Agent Starting Run: 175cm3od with config:
wandb: 	activation_fn: LeakyReLU
wandb: 	batch_size: 94
wandb: 	epochs: 100
wandb: 	latent_layer_size: 25
wandb: 	learning_rate: 0.0016400838929796135
wandb: 	optimizer: nadam
wandb: 	window_length: 90


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/100
700/702 [============================>.] - ETA: 0s - loss: 0.5912 - mse: 0.5912

INFO:tensorflow:Assets written to: C:\Users\ucesvpm\OneDrive - University College London\PhD Project\Data Analytics\Time Series Clustering\Third Test\Time-Series-PhD\wandb\run-20230302_105905-175cm3od\files\model-best\assets


INFO:tensorflow:Assets written to: C:\Users\ucesvpm\OneDrive - University College London\PhD Project\Data Analytics\Time Series Clustering\Third Test\Time-Series-PhD\wandb\run-20230302_105905-175cm3od\files\model-best\assets
wandb: Adding directory to artifact (C:\Users\ucesvpm\OneDrive - University College London\PhD Project\Data Analytics\Time Series Clustering\Third Test\Time-Series-PhD\wandb\run-20230302_105905-175cm3od\files\model-best)... Done. 0.2s


...

INFO:tensorflow:Assets written to: C:\Users\ucesvpm\OneDrive - University College London\PhD Project\Data Analytics\Time Series Clustering\Third Test\Time-Series-PhD\wandb\run-20230302_124939-g6aywcqx\files\model-best\assets
wandb: Adding directory to artifact (C:\Users\ucesvpm\OneDrive - University College London\PhD Project\Data Analytics\Time Series Clustering\Third Test\Time-Series-PhD\wandb\run-20230302_124939-g6aywcqx\files\model-best)... Done. 0.2s


3296/3296 [==============================] - 52s 16ms/step - loss: 0.0545 - mse: 0.0545 - val_loss: 0.0998 - val_mse: 0.0998 - lr: 6.6889e-05
Epoch 54/100
3296/3296 [==============================] - 51s 15ms/step - loss: 0.0523 - mse: 0.0523 - val_loss: 0.1012 - val_mse: 0.1012 - lr: 6.6889e-05
Epoch 57/100
3296/3296 [==============================] - 54s 16ms/step - loss: 0.0519 - mse: 0.0519 - val_loss: 0.1013 - val_mse: 0.1013 - lr: 6.6889e-05
Epoch 58/100
3296/3296 [==============================] - 60s 18ms/step - loss: 0.0514 - mse: 0.0514 - val_loss: 0.1018 - val_mse: 0.1018 - lr: 6.6889e-05
Epoch 59/100
2107/3296 [==================>...........] - ETA: 17s - loss: 0.0481 - mse: 0.0481

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



3296/3296 [==============================] - 63s 19ms/step - loss: 0.0471 - mse: 0.0471 - val_loss: 0.1020 - val_mse: 0.1020 - lr: 3.3444e-05


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
loss,█▆▅▅▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
mse,█▆▅▅▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
val_loss,█▇▆▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
val_mse,█▇▆▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
best_epoch,52
best_val_loss,0.09975
epoch,62
loss,0.04711
mse,0.04711
val_loss,0.10196


In [29]:
wandb.finish()

---
# Debugging